# Using lists to expand equations to many dimensions 

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:

from modelclass import model
from modelmanipulation import explode, doable_unroll,tofrml,explode_new
from model_latex_class import doable 
from modelpattern import list_extract


Before doable
<SUM=ABE,HEST> [BANKS=COUNTRY = DENMARK ] LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS}
do_indicies_dict={'BANKS': 'COUNTRY  =  DENMARK ', 'SECTORS': None}*******************


After doable
DO BANKS COUNTRY  =  DENMARK  $ DO SECTORS  $
  FRML    <SUM=ABE,HEST> LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS} $
ENDDO $ ENDDO $ 
FRML <SUM=ABE,HEST> LOSS__ABE = SUM(BANKS COUNTRY  =  DENMARK ,SUM(SECTORS,LOSS__{BANKS}__{SECTORS})) $ 




## LIST command, define list and sublists    

Lists are used to direct how the do command explodes a template model. 
And to define sum functions, and for matrix packing and unpacking. 

``` 
List <listname> = <sublistname0> : <sl00> <sl01> .. <sl0n> / 
                     <sublistname1> : <sl10> <sl11> .. <sl1n> /
                     .............................]
                     $
```
**There should always be the same number of elements in sublists**

Example with one sublist: 

```
LIST BANKLIST = BANK : ATBAW DEHAS DEHSH LUPCK DELBY DEVWF ESSAB  $
```

Example with two sublist: 

```
LIST CRCOLIST  =  CRCOUNTRY   : UK DE CH NL BE CA JP KY LU HK  / 
                  CRMODELGEO  : UK DE CH NL BE CA JP R20 LU HK $
``` 

If a list name is created several times, the first time is the used. This makes it more easy to make prototypes for a few banks.


## DO ...  ENDDO, loop over lists   

```
do <listname> $
stuff 
enddo $
```

Will take take the stuff and make a copy for each element of the sublists in list name. 

In [2]:
mtest = ''' 
list banklist = bank : ib soren $
do banklist $
     profit_{bank} = revenue_{bank} - expenses_{bank} $
enddo $ 
'''

print(explode_new(mtest))

LIST BANKLIST = BANK : IB SOREN $

PROFIT_IB = REVENUE_IB - EXPENSES_IB $
PROFIT_SOREN = REVENUE_SOREN - EXPENSES_SOREN $




### Use of sublist to inject additional information 

In [5]:
mtest = ''' 
list banklist = bank    : ib      soren  marie /
                country : denmark sweden denmark  $
                
do banklist $
    frml <> profit_{bank} = revenue_{bank} - expenses_{bank} $
    frml <> expenses_{bank} = factor_{country} * revenue_{bank} $
enddo $ 
'''

print(explode(mtest))

LIST BANKLIST = BANK    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  $
FRML <> PROFIT_IB = REVENUE_IB - EXPENSES_IB $
FRML <> EXPENSES_IB = FACTOR_DENMARK * REVENUE_IB $
FRML <> PROFIT_SOREN = REVENUE_SOREN - EXPENSES_SOREN $
FRML <> EXPENSES_SOREN = FACTOR_SWEDEN * REVENUE_SOREN $
FRML <> PROFIT_MARIE = REVENUE_MARIE - EXPENSES_MARIE $
FRML <> EXPENSES_MARIE = FACTOR_DENMARK * REVENUE_MARIE $


### Use of sublist to create conditional expressions 

In [32]:
mtest = ''' 
list banklist = bank    : ib      soren  marie /
                country : denmark sweden denmark  $
                
do banklist country = denmark$
    frml <> profit_{bank} = revenue_{bank} - expenses_{bank} $
    frml <> expenses_{bank} = factor_{country} * revenue_{bank} $
enddo $ 
'''

print(explode(mtest))

<
Model name                              :            testmodel 
Model structure                         :            Recursive 
Number of variables                     :                    7 
Number of exogeneous  variables         :                    3 
Number of endogeneous variables         :                    4 
>

### Nested do

Do loops can be nested.

One just have to take care that there is no duplication of sublist names. 

In [7]:
mtest = ''' 
list banklist = bank    : ib      soren  marie /
                country : denmark sweden denmark  $
                
list assetlist = asset  : nfc sme hh             $
                
                
do banklist $
  £ bank = {bank}
    profit_{bank} = revenue_{bank} - expenses_{bank}-loss_{bank} $
    expenses_{bank} = factor_{country} * revenue_{bank} $
    do assetlist $
      loss_{asset}_{bank} = 0.02 * stock_{asset}_{bank} $
    enddo $
    £ find the sum for {bank}
    loss_{bank} = sum(assetlist,loss_{asset}_{bank})    $

enddo $ 
'''

print(explode(mtest))

LIST BANKLIST = BANK    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  $
LIST ASSETLIST = ASSET  : NFC SME HH             $
£ BANK = IB
PROFIT_IB = REVENUE_IB - EXPENSES_IB-LOSS_IB $
EXPENSES_IB = FACTOR_DENMARK * REVENUE_IB $
LOSS_NFC_IB = 0.02 * STOCK_NFC_IB $
LOSS_SME_IB = 0.02 * STOCK_SME_IB $
LOSS_HH_IB = 0.02 * STOCK_HH_IB $
£ FIND THE SUM FOR IB
LOSS_IB = (LOSS_NFC_IB+LOSS_SME_IB+LOSS_HH_IB)    $
£ BANK = SOREN
PROFIT_SOREN = REVENUE_SOREN - EXPENSES_SOREN-LOSS_SOREN $
EXPENSES_SOREN = FACTOR_SWEDEN * REVENUE_SOREN $
LOSS_NFC_SOREN = 0.02 * STOCK_NFC_SOREN $
LOSS_SME_SOREN = 0.02 * STOCK_SME_SOREN $
LOSS_HH_SOREN = 0.02 * STOCK_HH_SOREN $
£ FIND THE SUM FOR SOREN
LOSS_SOREN = (LOSS_NFC_SOREN+LOSS_SME_SOREN+LOSS_HH_SOREN)    $
£ BANK = MARIE
PROFIT_MARIE = REVENUE_MARIE - EXPENSES_MARIE-LOSS_MARIE $
EXPENSES_MARIE = FACTOR_DENMARK * REVENUE_MARIE $
LOSS_NFC_MARIE = 0.02 * STOCK_NFC_MARIE $
LOSS_SME_MARIE = 0.02 * STOCK_SME_MARIE $
LOSS_HH_MARIE = 0.02 

### Use of sublist to do conditional Loop

By using 

`DO listname sublist = value $`

the looping will only be performed on the members of list where the value of the corresponding sublist match the value in the do statement.

This can be useful for instance when treating portfolios differently depending on some information. An example could be the different treatment of portfolios depending on regulatory approach. 

In [8]:
mtest='''
list portdic = port  : households ,  NFC ,   RE,  sov  /
               reg   :       AIRB , AIRB ,  STA,  STA $
    do portdic reg = airb $ 
    frml x {port}_REA_W = calc_rw({port}_pd ,{port}_lgd) $
    enddo $ 
    do portdic reg = sta $ 
    frml x {port}_REA_W = {port}_REA_W(-1) $
    enddo $ 
 '''
print(explode(mtest))

LIST PORTDIC = PORT  : HOUSEHOLDS ,  NFC ,   RE,  SOV  /
               REG   :       AIRB , AIRB ,  STA,  STA $
FRML X HOUSEHOLDS_REA_W = CALC_RW(HOUSEHOLDS_PD ,HOUSEHOLDS_LGD) $
FRML X NFC_REA_W = CALC_RW(NFC_PD ,NFC_LGD) $
FRML X RE_REA_W = RE_REA_W(-1) $
FRML X SOV_REA_W = SOV_REA_W(-1) $


### Dynamic defined lists to utilize sparsity
Often not all the potential dimensions in a model contains data. For instance all banks don't have positions in all potential countries. 

In order to speed up calculations and to avoid bloating dataframes, we want to avoid calculating and carrying a lot of zeros around. 

This can be achieved by using dynamic naming of lists. For instance create a separate list of countries for each bank. 

The list for each country can be created by using the pandas library to identify in which countries 
there are non-zero positions - straight forward but beyond the scope of this notebook. 

Below is an example where there are two banks which each has exposures in different countries. 

In [9]:
    stest='''
    list BANKDIC = bank : Danske , Nordea $                                 
    list country_danske_dic = country : uk , DK, IR $
    list country_nordea_dic = country: SE , DK $
    do bankdic $ 
        frml x {bank}_income = {bank}_a +{bank}_b $
        do country_{bank}_dic $
            frml x value_{bank}_{country} = 42 $
        enddo $ 
        frml <> total_{bank} = sum(country_{bank}_dic,value_{bank}_{country})  $
      enddo $ '''
    
    print(explode(stest))

LIST BANKDIC = BANK : DANSKE , NORDEA $
LIST COUNTRY_DANSKE_DIC = COUNTRY : UK , DK, IR $
LIST COUNTRY_NORDEA_DIC = COUNTRY: SE , DK $
FRML X DANSKE_INCOME = DANSKE_A +DANSKE_B $
FRML X VALUE_DANSKE_UK = 42 $
FRML X VALUE_DANSKE_DK = 42 $
FRML X VALUE_DANSKE_IR = 42 $
FRML <> TOTAL_DANSKE = (VALUE_DANSKE_UK+VALUE_DANSKE_DK+VALUE_DANSKE_IR)  $
FRML X NORDEA_INCOME = NORDEA_A +NORDEA_B $
FRML X VALUE_NORDEA_SE = 42 $
FRML X VALUE_NORDEA_DK = 42 $
FRML <> TOTAL_NORDEA = (VALUE_NORDEA_SE+VALUE_NORDEA_DK)  $


In [10]:
list_extract(mtest)

defaultdict(list,
            {'PORTDIC': defaultdict(list,
                         {'PORT': ['HOUSEHOLDS', 'NFC', 'RE', 'SOV'],
                          'REG': ['AIRB', 'AIRB', 'STA', 'STA'],
                          'PORT_END': ['0', '0', '0', '1'],
                          'PORT_NOEND': ['1', '1', '1', '0'],
                          'PORT_START': ['1', '0', '0', '0'],
                          'PORT_NOSTART': ['0', '1', '1', '1'],
                          'PORT_MIDDLE': ['0', '1', '1', '0'],
                          'PORT_BEFORE': ['0', 'HOUSEHOLDS', 'NFC', 'RE'],
                          'PORT_AFTER': ['NFC', 'RE', 'SOV', '0']})})

## Sum, sum over list 
sum(LIST,expression) = sums expression over the elements of a list substitution the sublist names.  

In [11]:
mtest = ''' 
list banklist = bank    : ib      soren  marie /
                country : denmark sweden denmark  $
                
list assetlist = asset  : nfc sme hh             $
                
do banklist $
    £
    £ bank {bank}
    do assetlist $
      loss_{asset}_{bank} = 0.02 * stock_{asset}_{bank} $
    enddo $
    loss_{bank} = sum(assetlist,loss_{asset}_{bank})   $
enddo $ 
£
£ sum for all banks 
£
loss_total =  sum(banklist,sum(assetlist,loss_{asset}_{bank}))$
'''

print(explode(mtest))

LIST BANKLIST = BANK    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  $
LIST ASSETLIST = ASSET  : NFC SME HH             $
£
£ BANK IB
LOSS_NFC_IB = 0.02 * STOCK_NFC_IB $
LOSS_SME_IB = 0.02 * STOCK_SME_IB $
LOSS_HH_IB = 0.02 * STOCK_HH_IB $
LOSS_IB = (LOSS_NFC_IB+LOSS_SME_IB+LOSS_HH_IB)   $
£
£ BANK SOREN
LOSS_NFC_SOREN = 0.02 * STOCK_NFC_SOREN $
LOSS_SME_SOREN = 0.02 * STOCK_SME_SOREN $
LOSS_HH_SOREN = 0.02 * STOCK_HH_SOREN $
LOSS_SOREN = (LOSS_NFC_SOREN+LOSS_SME_SOREN+LOSS_HH_SOREN)   $
£
£ BANK MARIE
LOSS_NFC_MARIE = 0.02 * STOCK_NFC_MARIE $
LOSS_SME_MARIE = 0.02 * STOCK_SME_MARIE $
LOSS_HH_MARIE = 0.02 * STOCK_HH_MARIE $
LOSS_MARIE = (LOSS_NFC_MARIE+LOSS_SME_MARIE+LOSS_HH_MARIE)   $
£
£ SUM FOR ALL BANKS 
£
LOSS_TOTAL =  ((LOSS_NFC_IB+LOSS_SME_IB+LOSS_HH_IB)+(LOSS_NFC_SOREN+LOSS_SME_SOREN+LOSS_HH_SOREN)+(LOSS_NFC_MARIE+LOSS_SME_MARIE+LOSS_HH_MARIE))$


## Sum, conditional sums 
sum(LIST,expression) = sums expression over the elements of a list substitution the sublist names.  

In [12]:
mtest = ''' 
list banklist = bank    : ib      soren  marie /
                country : denmark sweden denmark  $
                
list assetlist = asset  : nfc sme hh             $
                
do banklist $
    £
    £ bank {bank}
    do assetlist $
      loss_{asset}_{bank} = 0.02 * stock_{asset}_{bank} $
    enddo $
    loss_{bank} = sum(assetlist,loss_{asset}_{bank})   $
enddo $ 
£
£ sum for all banks 
£
loss_total__sweden =  sum(banklist country=sweden,sum(assetlist,loss_{asset}_{bank}))$
loss_total__denmark =  sum(banklist country=denmark,sum(assetlist,loss_{asset}_{bank}))$

'''

print(explode(mtest))

LIST BANKLIST = BANK    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  $
LIST ASSETLIST = ASSET  : NFC SME HH             $
£
£ BANK IB
LOSS_NFC_IB = 0.02 * STOCK_NFC_IB $
LOSS_SME_IB = 0.02 * STOCK_SME_IB $
LOSS_HH_IB = 0.02 * STOCK_HH_IB $
LOSS_IB = (LOSS_NFC_IB+LOSS_SME_IB+LOSS_HH_IB)   $
£
£ BANK SOREN
LOSS_NFC_SOREN = 0.02 * STOCK_NFC_SOREN $
LOSS_SME_SOREN = 0.02 * STOCK_SME_SOREN $
LOSS_HH_SOREN = 0.02 * STOCK_HH_SOREN $
LOSS_SOREN = (LOSS_NFC_SOREN+LOSS_SME_SOREN+LOSS_HH_SOREN)   $
£
£ BANK MARIE
LOSS_NFC_MARIE = 0.02 * STOCK_NFC_MARIE $
LOSS_SME_MARIE = 0.02 * STOCK_SME_MARIE $
LOSS_HH_MARIE = 0.02 * STOCK_HH_MARIE $
LOSS_MARIE = (LOSS_NFC_MARIE+LOSS_SME_MARIE+LOSS_HH_MARIE)   $
£
£ SUM FOR ALL BANKS 
£
LOSS_TOTAL__SWEDEN =  ((LOSS_NFC_SOREN+LOSS_SME_SOREN+LOSS_HH_SOREN))$
LOSS_TOTAL__DENMARK =  ((LOSS_NFC_IB+LOSS_SME_IB+LOSS_HH_IB)+(LOSS_NFC_MARIE+LOSS_SME_MARIE+LOSS_HH_MARIE))$


## Doable: A little more high level 

All the do .. enddo statements introduces some visual noise. 
To handle the situation, where we just want to loop over some indexes and do calculations the **doable** function can handle some of the details. It will extract the indicies from the left hand side and loop over these indicies. 
ALso if the option sum= is provided the the sums of all the resulting left hand sides will be calculated. 


In [31]:
frml = '''
list sectors = sectors : a b 
list banks   = banks : hest ko 

doable  loss_{banks}__{sectors} =holding__{banks}__{sectors} * pd__{banks}__{sectors}
FRML X = 42 
doable <sum= all>    b__{sectors}__{banks}  = b
doable <sum= all>    diff(xx__{sectors}__{banks})  = 42

        '''.upper()
frml_1 = (doable_unroll(tofrml(frml)))

print(f'''
Original equations:
------------------
{frml}\n
''')
      
print(f'''
DO injected:
------------
{frml_1}
''')
frml_2  = explode(frml_1)

print(f'''
Resulting Equations:
--------------------
\n  {frml_2}         
          ''')






Original equations:
------------------

LIST SECTORS = SECTORS : A B 
LIST BANKS   = BANKS : HEST KO 

DOABLE  LOSS_{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS}
FRML X = 42 
DOABLE <SUM= ALL>    B__{SECTORS}__{BANKS}  = B
DOABLE <SUM= ALL>    DIFF(XX__{SECTORS}__{BANKS})  = 42

        



DO injected:
------------
LIST SECTORS = SECTORS : A B $
LIST BANKS   = BANKS : HEST KO $
DO SECTORS  $
  FRML    <> LOSS_{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS}$ $
ENDDO $  
FRML X = 42 $
DO SECTORS  $ DO BANKS  $
  FRML    <SUM= ALL> B__{SECTORS}__{BANKS}  = B$ $
ENDDO $ ENDDO $ 
FRML <SUM= ALL> B__ALL = SUM(SECTORS,SUM(BANKS,B__{SECTORS}__{BANKS})) $ 

 
DO SECTORS  $ DO BANKS  $
  FRML    <SUM= ALL> DIFF(XX__{SECTORS}__{BANKS})  = 42$ $
ENDDO $ ENDDO $ 
FRML <SUM= ALL> XX__ALL = SUM(SECTORS,SUM(BANKS,XX__{SECTORS}__{BANKS})) $ 

 


Resulting Equations:
--------------------

  LIST SECTORS = SECTORS : A B $
LIST BANKS   = BANKS : HE

In [12]:
frml = '''loss_{sectors}=  42
        '''.upper()
frml_1 = (doable(frml))
frml_2  = explode(frml_1.upper() )

print(f'''\
Original equations:
------------------
{frml}\n
DO injected:
------------
{frml_1}
Resulting Equations:
--------------------
\n  {frml_2}         
          ''')


do_indicies_dict={}*******************
Original equations:
------------------
LOSS_{SECTORS}=  42
        

DO injected:
------------
FRML <> LOSS_{SECTORS}=  42 $
Resulting Equations:
--------------------

  FRML <> LOSS_{SECTORS}=  42 $         
          


## Pack values into matrices (a cvxopt class)

In [13]:
mtest2 = ''' 
list banklist  = bank : ib soren $
list banklist2 = bank2 : ib soren $
! vector 
frml <matrix> vbank = to_matrix(banklist,profit_{bank})  $
! matrices 
frml <matrix> Mbank = to_matrix(banklist,banklist2,loanfrom_{bank}_to_{bank2})  $
'''

print(explode(mtest2))

LIST BANKLIST  = BANK : IB SOREN $
LIST BANKLIST2 = BANK2 : IB SOREN $
VECTOR FRML <MATRIX> VBANK = MATRIX(
[PROFIT_IB,PROFIT_SOREN])  $
MATRICES FRML <MATRIX> MBANK = MATRIX(
[[LOANFROM_IB_TO_IB,LOANFROM_IB_TO_SOREN],
[LOANFROM_SOREN_TO_IB,LOANFROM_SOREN_TO_SOREN]])  $


## Pack values into arrays (a numpy class)

In [14]:
mtest2 = ''' 
list banklist  = bank : ib soren $
list banklist2 = bank2 : ib soren $
! vector 
frml <matrix> vbank = to_array(banklist,profit_{bank})  $
! matrices 
frml <matrix> Mbank = to_array(banklist,banklist2,loanfrom_{bank}_to_{bank2})  $
'''

print(explode(mtest2))

LIST BANKLIST  = BANK : IB SOREN $
LIST BANKLIST2 = BANK2 : IB SOREN $
VECTOR FRML <MATRIX> VBANK = ARRAY(
[PROFIT_IB,PROFIT_SOREN])  $
MATRICES FRML <MATRIX> MBANK = ARRAY(
[[LOANFROM_IB_TO_IB,LOANFROM_IB_TO_SOREN],
[LOANFROM_SOREN_TO_IB,LOANFROM_SOREN_TO_SOREN]])  $


## Unpack values - Argexpand 

In [15]:
mtest2 = ''' 
list banklist  = bank : ib soren $
list banklist2 = bank2 : ib soren $
! vector 
frml <> argexpand(banklist,test_{bank}) = to_matrix(banklist,profit_{bank})  $
! matrices 
frml <> argexpand(banklist,argexpand(banklist2,loanfrom_{bank2}_{bank})) = 0  $
'''

print(explode(mtest2))

LIST BANKLIST  = BANK : IB SOREN $
LIST BANKLIST2 = BANK2 : IB SOREN $
VECTOR FRML <> TEST_IB,TEST_SOREN = MATRIX(
[PROFIT_IB,PROFIT_SOREN])  $
MATRICES FRML <> LOANFROM_IB_IB,LOANFROM_SOREN_IB,LOANFROM_IB_SOREN,LOANFROM_SOREN_SOREN = 0  $


In [16]:
mtest2 = ''' 
list listdevs  = devs : dev_yer dev_uts $
list listshocks = shocks : ib soren $
    frml <matrix> ARGEXPAND(listdevs,{devs}) = STE(to_matrix(LISTSHOCKS,{SHOCKS}(-3)),\
    to_matrix(LISTSHOCKS,{SHOCKS}(-2 )), \
    to_matrix(LISTSHOCKS,{SHOCKS}(-1)),
    to_matrix(LISTSHOCKS,{SHOCKS})  ) $
    '''.upper() 

print(explode(mtest2))

LIST LISTDEVS  = DEVS : DEV_YER DEV_UTS $
LIST LISTSHOCKS = SHOCKS : IB SOREN $
FRML <MATRIX> DEV_YER,DEV_UTS = STE(MATRIX(
[IB(-3),SOREN(-3)]),    MATRIX(
[IB(-2 ),SOREN(-2 )]),     MATRIX(
[IB(-1),SOREN(-1)]),
    MATRIX(
[IB,SOREN])  ) $


## Create an array of lagged or leaded variables - lag_array
lag_array(<number>,<expression>) will create an numpy array with length = |number|, positive number 
    will give an array of leaded values, negative numbers an array of lagged values 

In [17]:
mtest3 = '''
pd_pit_array_leaded   = lag_array(3,pd_pit) $ 
'''
print(explode(mtest3))

PD_PIT_ARRAY_LEADED = ARRAY([PD_PIT(+1),PD_PIT(+2),PD_PIT(+3)]) $


In [18]:
mtest3 = '''
pd_pit_array_lagged   = lag_array(-3,pd_pit) $ 
'''
print(explode(mtest3))

PD_PIT_ARRAY_LAGGED = ARRAY([PD_PIT(-1),PD_PIT(-2),PD_PIT(-3)]) $


# Testing doable

In [134]:
from model_latex_class import doable

In [135]:
medsum = doable('<HEST,sum=abe> [banks=country=sweden,sektors=sektors]  LOSS__{BANKS}__{SECTORs} =HOLDING__{BANKS}__{SECTORs} * PD__{BANKS}__{SECTORs}'.upper() 


In [136]:
xx = doable('<HEST> [banks=country=denmark]  LOSS__{BANKS}__{SECTORs} =HOLDING__{BANKS}__{SECTORs} * PD__{BANKS}__{SECTORs}'.upper() 
    ,show=True)
#print(xx)


Before doable
<HEST> [BANKS=COUNTRY=DENMARK]  LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS}
do_conditions='BANKS=COUNTRY=DENMARK'
all_do_condition={'BANKS': 'COUNTRY = DENMARK'}
do_indicies_dict={'BANKS': 'COUNTRY = DENMARK', 'SECTORS': None}*******************


After doable
DO BANKS COUNTRY = DENMARK $ DO SECTORS  $
  FRML    <HEST> LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS} $
ENDDO $ ENDDO $ 


In [137]:
print(explode(lists+xx))

LIST BANKS = BANKS    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  /
                SELECTED : 1 0 1
                $
LIST SECTORS   = SECTORS  : NFC SME HH             $
FRML <HEST> LOSS__IB__NFC =HOLDING__IB__NFC * PD__IB__NFC $
FRML <HEST> LOSS__IB__SME =HOLDING__IB__SME * PD__IB__SME $
FRML <HEST> LOSS__IB__HH =HOLDING__IB__HH * PD__IB__HH $
FRML <HEST> LOSS__MARIE__NFC =HOLDING__MARIE__NFC * PD__MARIE__NFC $
FRML <HEST> LOSS__MARIE__SME =HOLDING__MARIE__SME * PD__MARIE__SME $
FRML <HEST> LOSS__MARIE__HH =HOLDING__MARIE__HH * PD__MARIE__HH $


In [180]:
extra = 'frml <x>   ialt2=      sum(banks country = denmark,sum(sectors,{banks}_{sectors}_income)) $ '
extra2 = 'FRML <SUM> LOSS__ABE = SUM(BANKS COUNTRY = DENMARK,SUM(SECTORS ,LOSS__{BANKS}__{SECTORS})) $ '
medsum = doable('<HEST,sum=abe> [banks=country=denmark,sektors=sektors]  LOSS__{BANKS}__{SECTORs} =HOLDING__{BANKS}__{SECTORs} * PD__{BANKS}__{SECTORs}'.upper() 
    ,show=False)
print(explode(lists+extra2))



LIST BANKS = BANKS    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  /
                SELECTED : 1 0 1
                $
LIST SECTORS   = SECTORS  : NFC SME HH             $
FRML <SUM> LOSS__ABE = ((LOSS__IB__NFC+LOSS__IB__SME+LOSS__IB__HH)+(LOSS__MARIE__NFC+LOSS__MARIE__SME+LOSS__MARIE__HH)) $


In [175]:
extra = '    frml <x> ialt2=sum(banks country=denmark,sum(sectors,{banks}_{sectors}_income)) $ '

medsum = doable('<HEST,sum=abe> [banks=country=sweden,sektors=sektors]  LOSS__{BANKS}__{SECTORs} =HOLDING__{BANKS}__{SECTORs} * PD__{BANKS}__{SECTORs}'.upper() 
    ,show=False)
print(explode(lists+medsum))



LIST BANKS = BANKS    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  /
                SELECTED : 1 0 1
                $
LIST SECTORS   = SECTORS  : NFC SME HH             $
FRML <HEST,SUM=ABE> LOSS__SOREN__NFC =HOLDING__SOREN__NFC * PD__SOREN__NFC $
FRML <HEST,SUM=ABE> LOSS__SOREN__SME =HOLDING__SOREN__SME * PD__SOREN__SME $
FRML <HEST,SUM=ABE> LOSS__SOREN__HH =HOLDING__SOREN__HH * PD__SOREN__HH $
FRML <HEST,SUM=ABE> LOSS__ABE = ((LOSS__SOREN__NFC+LOSS__SOREN__SME+LOSS__SOREN__HH)) $


In [69]:
yy = '''do BANKS COUNTRY  = SWEDEN  $ do SECTORS  $
  frml    <> LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS} $
enddo $ enddo $ '''
print(explode(lists+yy))

LIST BANKS = BANKS    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  /
                SELECTED : 1 0 1
                $
LIST SECTORS   = SECTORS  : NFC SME HH             $
FRML <> LOSS__SOREN__NFC =HOLDING__SOREN__NFC * PD__SOREN__NFC $
FRML <> LOSS__SOREN__SME =HOLDING__SOREN__SME * PD__SOREN__SME $
FRML <> LOSS__SOREN__HH =HOLDING__SOREN__HH * PD__SOREN__HH $


In [145]:
    from modelmanipulation import sumunroll, dounloop
    print(sumunroll(dounloop(
    '''
    LIST BANKS = BANK    : IB      SOREN  MARIE /
                COUNTRY : DENMARK SWEDEN DENMARK  /
                selected : 1 0 1
                $
LIST sectorS   = sectors  : NFC SME HH             $
    do BANKs $ 
       frml <x> {bank}_income = {bank}_a +{bank}_b $
    enddo $ 
    frml <x> ialt=sum(banks country=denmark,{bank}_income) $
    frml <x> ialt2=sum(banks country=denmark,sum(sectors,{bank}_{sectors}_income)) $ 

    ''')))  


LIST BANKS = BANK    : IB      SOREN  MARIE /
            COUNTRY : DENMARK SWEDEN DENMARK  /
            SELECTED : 1 0 1
            $
LIST SECTORS   = SECTORS  : NFC SME HH             $
FRML <X> IB_INCOME = IB_A +IB_B $
FRML <X> SOREN_INCOME = SOREN_A +SOREN_B $
FRML <X> MARIE_INCOME = MARIE_A +MARIE_B $
FRML <X> IALT=(IB_INCOME+MARIE_INCOME) $
FRML <X> IALT2=((IB_NFC_INCOME+IB_SME_INCOME+IB_HH_INCOME)+(MARIE_NFC_INCOME+MARIE_SME_INCOME+MARIE_HH_INCOME)) $
